# Imports

In [3]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from __future__ import print_function

import glob
import math
import os

import numpy as np
import pandas as pd


from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer 
import tensorflow as tf

import keras
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, Lambda, MaxPooling2D
from keras import backend as K
from keras import layers
from keras.utils.np_utils import to_categorical
from keras import Model
from keras.preprocessing.text import one_hot
from keras import regularizers
from keras import optimizers

from tensorflow.keras.optimizers import RMSprop
from tensorflow.python.data import Dataset

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

ModuleNotFoundError: No module named 'tensorflow'

# Classes

In [9]:
class NN:
    def __init__(self, input_shape, units):
        """
        units: (10, 11, 12)
            => 3 couches avec 10 neurones pour la premiere
        """
        nb_output = input_shape
        self.weights = []
        for nb_neurones in units:
            self.weights.append(np.random.rand(nb_output, nb_neurones))
            self.weights.append(np.zeros(nb_neurones))
            nb_output = nb_neurones
    
    def print_weights(self):
        for l in self.weights:
            print(l)

    def preactiv(self, x):
        res = x
        for i in range(0, len(self.weights), 2):
            res = self.activ(res @ self.weights[i] + self.weights[i + 1])

        return res

    def activ(self, res):
        res[res < 0] *= 0.2
        return res

    def predict(self, x):
        return self.activ(self.preactiv(x))

    def predict_on_dataset(self, x):
        res = np.zeros(len(x))
        for i in range(len(x)):
            res[i] = self.predict(x.iloc[i])
        return res

    def loss(self, y, res):  # 1 example
        return res - y

    def d_activation(self, res):
        res2 = res
        res2[res < 0] = 0.2
        res2[res >= 0] = 1
        return res2
    
    def cost(self, x, y):  # dataset
        total_cost = 0

        for i in range(len(x)):
            features = x.iloc(i)
            target = y.iloc(i)
            total_cost += (target - self.predict(features)) ** 2

        return total_cost

    def fit(self, x, y, step, epochs, batch_size, learning_rate, validation_datas):
        x_val = validation_datas[0]
        y_val = validation_datas[1]
        x = x
        y = y
        i = 0
        epo = 0
        steps_per_epoch = step // epochs

        default_batch_gradient = []
        for k in range(0, len(self.weights), 2):
            default_batch_gradient.append(np.zeros(self.weights[k].shape))
            default_batch_gradient.append(np.zeros(self.weights[k + 1].shape))

        # print("default_batch_gradient shape: ")

        # for q in range(len(default_batch_gradient)):
        #    print(default_batch_gradient[q].shape)

        for etape in tqdm(range(step)):
            i += 1
            bx = x.iloc[batch_size * i : batch_size * (i + 1) - 1]
            by = y.iloc[batch_size * i : batch_size * (i + 1) - 1]

            batch_gradient = default_batch_gradient # store the gradient calculated on the whole batch

            for k in range(len(bx)):  # iterate over the batch
                res_l = [bx.iloc[k]]  # store the result after each layer
                res = bx.iloc[k]
                for h in range(0, len(self.weights), 2):  # calculate the result for the current features
                    pre_a = res @ self.weights[h] + self.weights[h + 1]
                    res_l.append(pre_a)  # and store the intermediate result before activation
                    res = self.activ(pre_a)

                loss = self.loss(by.iloc[k], res)
                
                # update of the last neurone
                act_deriv = self.d_activation(res)
                batch_gradient[-2] += self.activ(np.expand_dims(res_l[-2], 1)) * loss
                batch_gradient[-1] += loss * act_deriv
                
                for b in reversed(range(0, len(self.weights) - 2, 2)):  # backpropagation
                    l_output = res_l[(b // 2) + 1]  # layer output
                    loss = self.activ(l_output) * np.dot(loss, self.weights[b + 2].T)
                    batch_gradient[b] = l_output * loss * self.d_activation(l_output)
#                     print(batch_gradient[b])
                    batch_gradient[b + 1] = l_output
                    # print(l_output * loss)

            for k in range(0, len(self.weights), 2):
                self.weights[k] -= learning_rate * (1 / batch_size) * batch_gradient[k]
                self.weights[k + 1] -= learning_rate * (1 / batch_size) * batch_gradient[k + 1]

            if x.shape[0] < batch_size * (i + 1):
                p = np.random.permutation(len(x))
                x = x.iloc[p]
                y = y.iloc[p]
                i = 0

            if etape != 0 and etape % (steps_per_epoch) == 0:
                print("=================")
                predic_val = self.predict_on_dataset(x_val)
#                 predic_train = self.predict_on_dataset(bx)
                predic_train = self.predict_on_dataset(x.iloc[i : i + (step // epochs)])
                # print("predic_train: ", predic_train)
                print(
                    "Epoch: ",
                    epo,
                    "| loss val: ",
                    ((predic_val - y_val) ** 2).mean(),
                    " | loss train ",
                    ((predic_train - y.iloc[i : i + (step // epochs)]) ** 2).mean() / 2,
                )
                print("mean predic: ", predic_val.mean())
                for l in self.weights:
#                     print(l)
                    pass
                epo += 1

# Recuperation des donnees

In [10]:
# dataset = pd.read_csv("./winequality-red.csv")

# train = dataset
# validation = dataset.tail(199)

# x_train = train.drop('quality', 1)
# y_train = train.quality

# train_sata = pd.read_csv("./train_satander.csv")
# x_train = train_sata.drop("target", 1)
# x_train = x_train.drop("ID_code", 1)
# y_train = train_sata.target

cali_dataframe = pd.read_csv("./california_housing_train.csv")
x_train = cali_dataframe.drop("median_house_value", 1)
y_train = cali_dataframe.median_house_value / 100000

# Normalisation et transformation

In [11]:
def normalize(x):
    return (x - min(x)) / (max(x) - min(x))


for x in x_train:
    x_train[x] = normalize(x_train[x])
    pass

x_val = x_train.tail(1000)
y_val = y_train.tail(1000)
print(x_val.describe())

print("y summary")
print(y_train.describe())

         longitude     latitude  housing_median_age  total_rooms  \
count  1000.000000  1000.000000         1000.000000  1000.000000   
mean      0.149141     0.652423            0.601255     0.064603   
std       0.053452     0.118337            0.265878     0.040652   
min       0.000000     0.527099            0.019608     0.000158   
25%       0.143426     0.556854            0.352941     0.040464   
50%       0.170319     0.618491            0.588235     0.056715   
75%       0.185259     0.691817            0.862745     0.076690   
max       0.190239     1.000000            1.000000     0.340055   

       total_bedrooms   population   households  median_income  
count     1000.000000  1000.000000  1000.000000    1000.000000  
mean         0.075392     0.032423     0.073566       0.228091  
std          0.045796     0.021597     0.046360       0.122239  
min          0.000000     0.000280     0.000000       0.000000  
25%          0.046671     0.019823     0.046210       0.147688

# Training

In [ ]:
input = layers.Input(shape=(80,))

x = layers.BatchNormalization()(input)
x = layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01))(input)
x = layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01))(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.25)(x)
x = layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01))(x)
x = layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01))(x)
x = layers.Dropout(0.25)(x)
# Create output layer with a single node and sigmoid activation
output = layers.Dense(1, kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01))(x)
model = Model(input, output)

In [12]:
model = NN(x_train.shape[1], (10, 10, 1))
print("first predic: ", model.predict(x_train.iloc[0]))

# print("starting weights")
# model.print_weights()

print("Training")
model.fit(
    x_train,
    y_train,
    step=10000,
    epochs=20,
    batch_size=20,
    learning_rate=0.00000005,
    validation_datas=(x_val, y_val),
)

  0%|          | 7/10000 [00:00<02:33, 64.95it/s]

first predic:  [23.83220874]
Training


  5%|▍         | 491/10000 [00:05<01:34, 100.36it/s]

  5%|▌         | 518/10000 [00:05<02:41, 58.73it/s] 

Epoch:  0 | loss val:  7.546738303633252  | loss train  1.754374815202586
mean predic:  -0.10501968050867089


...

Epoch:  17 | loss val:  16.17861008901815  | loss train  6.997349270155349
mean predic:  -1.438687996404754


 95%|█████████▍| 9496/10000 [01:48<00:04, 109.83it/s]

 95%|█████████▌| 9519/10000 [01:48<00:06, 71.33it/s] 

Epoch:  18 | loss val:  11.11972590882077  | loss train  4.596493753462482
mean predic:  -0.727572784935609


100%|██████████| 10000/10000 [01:52<00:00, 88.51it/s]
